# Finetune `meta-llama/Meta-Llama-3-8B-Instruct` on an EC2 instance using `Unsloth`
---

Unsloth makes finetuning large language models like Llama-3, Mistral, Phi-4 and Gemma 2x faster, use 70% less memory, and with no degradation in accuracy!

**Note**: ***This notebook is run on a `g6e.12xlarge` instance. Follow the prerequisite steps [here](README.md)***

In this example, we will be fine tuning the llama3 8b instruct model. There are several 4bit pre quantized models that `unsloth` provides that are not gated. This supports 4x faster downloading with no OOMs. In this case, we will be using the standard `meta-llama/Meta-Llama-3-8B-Instruct` model from hugging face. 

In [109]:
import os
import logging
import globals as g
from dotenv import load_dotenv
from unsloth import to_sharegpt
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import standardize_sharegpt
from ec2_metrics import EC2MetricsCallback

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [110]:
# Load environment variables from .env file
import getpass
load_dotenv()
if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your HuggingFace token: ")
hf_token = os.getenv("HF_TOKEN")

if not os.getenv("HF_MODEL_ID"):
    hf_model_id  = input("Enter the model id to use for fine-tuning (e.g. meta-llama/Llama-3.1-8B-Instruct): ")
else:
    hf_model_id = os.getenv("HF_MODEL_ID")
logger.info(f"hf_model_id={hf_model_id}")


[2025-03-13 15:48:47,036] p5203 {2478216038.py:12} INFO - hf_model_id=meta-llama/Llama-3.1-8B-Instruct


In [111]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

DATASET_OF_INTEREST: str = 'PolyAI/banking77'

ALPACA_PROMPT: str = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [112]:
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = hf_model_id,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = hf_token # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
except Exception as e:
    logger.error(f"Error occurred while loading the model: {e}")
    raise

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [01:46<00:00, 26.69s/it]


In [151]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


## Data Prep

We now use the Alpaca dataset from vicgalle, which is a version of 52K of the original Alpaca dataset generated from GPT4. You can replace this code section with your own data prep.

In [114]:
dataset = load_dataset(DATASET_OF_INTEREST, split="train", trust_remote_code=True)
logger.info(f"Columns in the dataset: {dataset.column_names}")

[2025-03-13 15:50:40,163] p5203 {1817604543.py:2} INFO - Columns in the dataset: ['text', 'label']


In [115]:
dataset = dataset.map(lambda x: {**x, "instructions": ""})

In [116]:
dataset

Dataset({
    features: ['text', 'label', 'instructions'],
    num_rows: 10003
})

In [117]:
dataset = to_sharegpt(
    dataset,
    merged_prompt="{text}[[\nYour input is:\n{instructions}]]",
    output_column_name="label",
    conversation_extension=3,
)

In [118]:
# Use the standardize_sharegpt function to just make the dataset in a correct format for finetuning
dataset = standardize_sharegpt(dataset)

In [119]:
from pprint import pprint
pprint(dataset[:3])

{'conversations': [[{'content': 'I am still waiting on my card?',
                     'role': 'user'},
                    {'content': '11', 'role': 'assistant'},
                    {'content': 'How can I convert currencies?',
                     'role': 'user'},
                    {'content': '33', 'role': 'assistant'},
                    {'content': 'Will be Apple Watch be able to let me top up?',
                     'role': 'user'},
                    {'content': '2', 'role': 'assistant'}],
                   [{'content': "What can I do if my card still hasn't arrived "
                                'after 2 weeks?',
                     'role': 'user'},
                    {'content': '11', 'role': 'assistant'},
                    {'content': 'Hello. Can you help figure out why the recent '
                                "transfer from my UK bank account isn't "
                                'showing up?',
                     'role': 'user'},
                    {'con

In [120]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


In [121]:
dataset[0]['conversations']

[{'content': 'I am still waiting on my card?', 'role': 'user'},
 {'content': '11', 'role': 'assistant'},
 {'content': 'How can I convert currencies?', 'role': 'user'},
 {'content': '33', 'role': 'assistant'},
 {'content': 'Will be Apple Watch be able to let me top up?', 'role': 'user'},
 {'content': '2', 'role': 'assistant'}]

In [204]:
%%time
# train the model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 600,
        max_steps = 600,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    callbacks=[EC2MetricsCallback],
)

Tokenizing train dataset (num_proc=2): 100%|██████████| 10003/10003 [00:00<00:00, 12901.26 examples/s]

CPU times: user 636 ms, sys: 247 ms, total: 884 ms
Wall time: 4.1 s


In [ ]:
%%time
# this will initiate the training process and also log the EC2 utilization metrics, such as the GPU
# utilization, CPU utilization, etc.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,003 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 600
 "-____-"     Number of trainable parameters = 41,943,040
[2025-03-13 16:31:41,297] p5203 {ec2_metrics.py:184} INFO - Training started. Initiating EC2 metrics collection.
[2025-03-13 16:31:41,298] p5203 {ec2_metrics.py:170} INFO - Writing header: ['timestamp', 'cpu_percent_mean', 'memory_percent_mean', 'memory_used_mean', 'gpu_utilization_mean', 'gpu_memory_used_mean', 'gpu_memory_free_mean', 'gpu_memory_total_mean']
[2025-03-13 16:31:41,299] p5203 {ec2_metrics.py:41} INFO - Starting collection
[2025-03-13 16:31:41,300] p5203 {ec2_metrics.py:143} INFO - Starting daemon collector to run in background


Step,Training Loss
1,0.748400
2,0.664100
3,0.803300
4,0.686400
5,0.762800
6,0.581500
7,0.510200
8,0.693100
9,0.564400
10,0.708600


[2025-03-13 16:45:10,774] p5203 {ec2_metrics.py:191} INFO - Training ended. Stopping EC2 metrics collection.
[2025-03-13 16:45:10,775] p5203 {ec2_metrics.py:33} INFO - Stopped collection


CPU times: user 11min 37s, sys: 2min 13s, total: 13min 50s
Wall time: 13min 31s


[2025-03-13 16:45:11,351] p5203 {ec2_metrics.py:33} INFO - Stopped collection


### Log the trainer stats
---

In this step, we log some of the trainer stats, such as the number of global steps it took to get to a specific training loss, the train runtime, samples per second, steps per second, etc.

In [206]:
# Format the training stats in a readable way
output_text = f"""Training Statistics:
Global Steps: {trainer_stats.global_step}
Training Loss: {trainer_stats.training_loss:.4f}

Metrics:
- Train Runtime: {trainer_stats.metrics['train_runtime']:.3f} seconds
- Training Samples/Second: {trainer_stats.metrics['train_samples_per_second']:.3f}
- Training Steps/Second: {trainer_stats.metrics['train_steps_per_second']:.3f}
- Total FLOPS: {trainer_stats.metrics['total_flos']:.2e}
- Final Train Loss: {trainer_stats.metrics['train_loss']:.4f}
"""

# Save to a text file
with open(os.path.join(g.RESULTS_DIR, g.TRAINING_STATS), 'w') as f:
    f.write(output_text)

In [207]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "I see a charge on my credit card statement but I paid on time, why?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
model = model.to("cuda")
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

63<|eot_id|>


In [217]:
question_list = ['I see a charge on my credit card statement but I paid on time, why?',
'Do you have a branch in Timbuktu?',
'I lost my card and my replacement card has not arrived.']

output_file = "problem1_task1.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for i in question_list:
        messages = [                    
            {"role": "user", "content": i},
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        # Move model to CUDA
        model = model.to("cuda")

        # Generate output
        output_ids = model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
        response_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

        # Prepare output text
        output_str = f"input: {i}\ncategory: {response_text}\n\n"

        # Print and save to file
        print(output_str)
        f.write(output_str)

input: I see a charge on my credit card statement but I paid on time, why?
category: 53


input: Do you have a branch in Timbuktu?
category: 24


input: I lost my card and my replacement card has not arrived.
category: 11


